## MPO HGA Blockage

This is a Python Jupyter Notebook to illustrate how to visualize the blockage that suffers the High Gain Antenna due to the MPO structure. 

First thing we will do is indicate that we want the Python package matplotlib to be output in the notebook and to import the SpiceyPy package to use SPICE. We also ensure that project_path module is include, this will allow us get access to common packages and resources

In [ ]:
from project_path import data_path
from ess.utils import mask_blockage
import spiceypy as cspice
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

We will setup the kernel set for the spice library, that will be used along the notebook. Note that if you are in **Datalabs** infrastructure and the SPICE volume is mounted, the latest kernel repositories are available. Additionally the *ess.datalabs* package contains many helpers and shortcuts.

Using the *data_path* shortcut we can get resources stored in the **ess-jupyternb/data** folder

In [ ]:
from ess.datalabs import get_local_metakernel, is_spice_volume_mounted

cspice.kclear()
if is_spice_volume_mounted():
    cspice.furnsh(get_local_metakernel('BEPICOLOMBO', 'bc_plan.tm'))
else:
    cspice.furnsh('/local/path/kernel')

cspice.furnsh(
    os.path.join(data_path, 'test', 'bc_mpo_hga_schulte_vector_test_v01.bc'))
cspice.furnsh(
    os.path.join(data_path, 'test', 'bc_mpo_sc_bus_v03.bds'))

In [ ]:



def rtpairs(r, n):
    for i in range(len(r)):
       for j in range(n[i]):
        yield r[i], j*(2 * np.pi / n[i])


time_step_seconds = 10 # time step
et0 = cspice.utc2et('2021-01-01T00:00:00')
etf = cspice.utc2et('2021-01-01T00:01:00')
times = np.linspace(et0, etf, int((etf - et0) / time_step_seconds))
el_angles = np.arange(-5.0, 145, 1)
az_angles = np.arange(-180, 181, 1)
coverage_map = np.zeros((el_angles.shape[0], az_angles.shape[0]))
et = et0
output = []
output_b = []
T = [1, 5, 10, 15, 20, 30, 40]
R = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
row_counter = el_angles.shape[0]
for i in range(el_angles.shape[0]):
    for j in range(az_angles.shape[0]):
        el = el_angles[i]
        az = az_angles[j]
        rs = []
        ds = []
        for r, t in rtpairs(R, T):
            r0 = np.array([r * np.cos(t), r * np.sin(t), 0]) / 1000
            r_opt = cspice.spkpos('MPO_HGA_OPT_EL', et, 'MPO_SPACECRAFT', 'NONE', 'MPO_SPACECRAFT')[0]
            M = cspice.pxform('MPO_HGA', 'MPO_SPACECRAFT', et)
            r = cspice.mxv(M, r0) + r_opt
            d = cspice.mxv(M, [0, 0, 1])
            rs.append(r)
            ds.append(d)
            
        xarray, flag_array = cspice.dskxv(False, 'MPO_SPACECRAFT', [-121000], et, 'MPO_SPACECRAFT',
                         rs, ds)
        
        coverage_map[i, j] += len(list(filter(lambda x: x, flag_array)))           
        et += time_step_seconds
    print(f'Row {i}/{row_counter}') 

plt.contourf(range(-180, 181), range(-5, 145), coverage_map, 100)
plt.colorbar()
plt.grid()
plt.show()

In [ ]:
coverage = coverage_map / np.sum(T) * 100
coverage[coverage == 0] = np.nan

fig = plt.figure(figsize=(14, 6))
plt.contourf(range(-180, 181), range(-5, 145), coverage, 20, alpha=0.1)
contour_lines = plt.contour(range(-180, 181), range(-5, 145), coverage, 20)
plt.clabel(contour_lines, contour_lines.levels, inline=True, fmt='%1.0f', fontsize=8)
plt.colorbar()
plt.ylim([90, 144])
plt.grid()
plt.ylabel('El [deg]')
plt.xlabel('Az [deg]')
plt.title('HGA disk percentage blockage')
plt.show()

In [ ]:
el_angles = np.arange(-5.0, 145, 1)
az_angles = np.arange(-180, 181, 1)
et = et0
output = []
output_b = []
for el in el_angles:
    for az in az_angles:
        for n in range(0, 360, 12):
            n = np.deg2rad(n)
            r0 = np.array([np.cos(n), np.sin(n), 0]) * 0.0005
            r_opt = cspice.spkpos('MPO_HGA_OPT_EL', et, 'MPO_SPACECRAFT', 'NONE', 'MPO_SPACECRAFT')[0]
            M = cspice.pxform('MPO_HGA', 'MPO_SPACECRAFT', et)
            r = cspice.mxv(M, r0) + r_opt
            d = cspice.mxv(M, [0, 0, 1])

            xarray, flag = cspice.dskxv(False, 'MPO_SPACECRAFT', [-121000], et, 'MPO_SPACECRAFT',
                             [r], [d])
            if flag == True:
                blocked = 1
                output_b.append([blocked, el, az, et])
            else:
                blocked = 0
            # try:
            #     cspice.sincpt('DSK/UNPRIORITIZED', 'MPO_SPACECRAFT', et, 'MPO_SPACECRAFT',
            #                   'NONE', 'MPO_HGA_OPT_EL', 'MPO_HGA', np.array([0, 0, 1]))
            #     blocked = 1
            #     output_b.append([blocked, el, az, et])
            # except:
            #     blocked = 0
        output.append([blocked, el, az, et])
        et += 10
print(len(output))
output_b = np.asarray(output_b)
plt.scatter(output_b[:, 2], output_b[:, 1], color='red')
plt.grid()
plt.xlim([-180, 180])
plt.ylim([-5, 145])
plt.show()

In [ ]:
fig = plt.figure(figsize=(14, 6))

plt.scatter(output_b[:, 2], output_b[:, 1], color='lavender')
plt.xlim([-180, 180])
plt.ylim([-5, 145])

plt.contourf(range(-180, 181), range(-5, 145), coverage, 20, alpha=0.05)
contour_lines = plt.contour(range(-180, 181), range(-5, 145), coverage, 20)
plt.clabel(contour_lines, contour_lines.levels, inline=True, fmt='%1.0f', fontsize=12)
plt.colorbar()
plt.ylim([90, 144])
plt.grid()
plt.ylabel('El [deg]')
plt.xlabel('Az [deg]')
plt.title('HGA disk percentage blockage')
plt.show()